In [3]:
!pip install  scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366548 sha256=ac53e99a8ec8e5b339543708472bf2a63ae37bf27031db13b728f39f63a7ca4f
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [126]:
from surprise import accuracy, Dataset, SVD, Reader, SVDpp, NMF
from surprise.model_selection import train_test_split
import pandas as pd

df = pd.read_csv('test.csv')
df['rating'] = 1
df_train, df_test = df.loc[:len(df)/2], df.loc[len(df)/2:]
reader = Reader(rating_scale=(1, 1))

# sample random trainset and testset
# test set is made of 25% of the ratings.
train_data = Dataset.load_from_df(df_train, reader)
trainset, testset_1 = train_test_split(train_data, test_size=0.5)


test_data = Dataset.load_from_df(df_test, reader)
testset, testset______ = train_test_split(train_data, test_size=0.25)

In [123]:
model = SVD(n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.02)
model.fit(trainset)
predictions = model.test(testset______)
preds = pd.DataFrame(predictions)
from tqdm.auto import tqdm
df_eval = pd.DataFrame(columns = ['uid', 'facts', 'preds'])
c = 0
user_set = df_test['user_id'].unique()
for user in tqdm(preds['uid'].unique()[:2000]):
    if user in user_set:
        df_eval.loc[c] = [
            user,
            list(df_test[df_test['user_id'] == user]['song'].values),
            list(preds[preds['uid'] == user]['iid'].values)            
                        ]
        c += 1
k = 20 
for user in tqdm(df_eval['uid'].unique()):
    true_count = 0
    for i in range(k):
        try:
            if df_eval[df_eval['uid'] == user]['facts'].values[0][i] == df_eval[df_eval['uid'] == user]['preds'].values[0][i]:
                true_count += 1
        except IndexError:
            continue
    df_eval.loc[
        df_eval[df_eval['uid'] == user].index,f'precision@{k}'
        ] = true_count/k




        # len(
        #     set(df_eval[df_eval['uid'] == user]['facts'].values[0]).intersection(set(df_eval[df_eval['uid'] == user]['preds'].values[0]))
        #     ) / len(
        #         set(df_eval[df_eval['uid'] == user]['facts'].values[0])
        #     )
df_eval[f"precision@{k}"].mean()

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/1983 [00:00<?, ?it/s]

0.00030257186081694405

In [134]:
dict_result = {}
from surprise import SVD, SVDpp, NMF, KNNBaseline, CoClustering
from tqdm.auto import tqdm
for model in [SVD(), SVDpp(), NMF(), KNNBaseline(), CoClustering()]:
    print(str(model))
    model.fit(trainset)
    predictions = model.test(testset______)
    preds = pd.DataFrame(predictions)

    df_eval = pd.DataFrame(columns = ['uid', 'facts', 'preds'])
    c = 0
    user_set = df_test['user_id'].unique()
    for user in tqdm(preds['uid'].unique()[:1000]):
        if user in user_set:
            df_eval.loc[c] = [
                user,
                list(df_test[df_test['user_id'] == user]['song'].values),
                list(preds[preds['uid'] == user]['iid'].values)            
                            ]
            c += 1
    k = 20 
    for user in tqdm(df_eval['uid'].unique()):
        true_count = 0
        for i in range(k):
            try:
                if df_eval[df_eval['uid'] == user]['facts'].values[0][i] == df_eval[df_eval['uid'] == user]['preds'].values[0][i]:
                    true_count += 1
            except IndexError:
                continue

        df_eval.loc[
            df_eval[df_eval['uid'] == user].index,f'precision@{k}'
            ] = true_count/k

        df_eval.loc[
            df_eval[df_eval['uid'] == user].index,f'myprecision'
            ] =  len(
                set(df_eval[df_eval['uid'] == user]['facts'].values[0]).intersection(set(df_eval[df_eval['uid'] == user]['preds'].values[0]))
                ) / len(
                     set(df_eval[df_eval['uid'] == user]['facts'].values[0])
                )
        


            # len(
            #     set(df_eval[df_eval['uid'] == user]['facts'].values[0]).intersection(set(df_eval[df_eval['uid'] == user]['preds'].values[0]))
            #     ) / len(
            #         set(df_eval[df_eval['uid'] == user]['facts'].values[0])
            #     )
    dict_result[str(model)] = (df_eval[f"precision@{k}"].mean(),
                               df_eval[f"myprecision"].mean())

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/996 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/996 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/996 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/996 [00:00<?, ?it/s]

In [135]:
dict_result

{'<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7f75415588e0>': (5.020080321285141e-05,
  0.009570723172815931),
 '<surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x7f7541558df0>': (5.020080321285141e-05,
  0.009570723172815931),
 '<surprise.prediction_algorithms.matrix_factorization.NMF object at 0x7f7541558b50>': (5.020080321285141e-05,
  0.009570723172815931),
 '<surprise.prediction_algorithms.knns.KNNBaseline object at 0x7f7541558d00>': (5.020080321285141e-05,
  0.009570723172815931),
 '<surprise.prediction_algorithms.co_clustering.CoClustering object at 0x7f753866c880>': (5.020080321285141e-05,
  0.009570723172815931)}